## Dependencies

In [1]:
# Confirmed needed dependencies
import zillow
import random
import pprint

# Seth 

# Research how to make this truly invisible on GitHub Repo
# from config import zwsid
# Paste in your zwsid in the meantime using the following line
# zwsid = ""

In [2]:
# Possibly needed, but not yet used dependencies
import pandas as pd
import requests
import json

/Users/troybailey/anaconda3/envs/pythonData/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/troybailey/anaconda3/envs/pythonData/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [4]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

# dependencies for geocoordinates generator
import random
import sys
import math

In [5]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 200        #### Set number of lat,lon points to generate

output_file_name = "random_Lat_Lon.csv" #### Set output file name 

In [6]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points = points+1

In [7]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

,lat,lon
0,30.322671,-97.702210
1,30.405773,-97.800212
2,30.292095,-97.746324
3,30.317465,-97.686401
4,30.411218,-97.747470


In [8]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(output_file_name, index=False, header=True)

## *TODO: Find a way to convert coordinates to residential addresses

In [ ]:
# Yuta

#Play with found sources to see if they will work

# Possible sources
# https://stackoverflow.com/questions/11390392/return-individual-address-components-city-state-etc-from-geopy-geocoder

# https://wiki.openstreetmap.org/wiki/Nominatim

In [ ]:
# Kat's block

## Zillow API Calls using Address and Zipcode

In [22]:
address = "929 Cavalier Ln" 
zipcode = "78757"

pp = pprint.PrettyPrinter(indent=4)

api = zillow.ValuationApi()


#This appears to be the ideal search to use with the most flushed out info
deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
pp.pprint(deep_results.get_dict())


# This appears to return the same as .GetZEstimate(),
# but has less accurate "extended_data" than .GetDeepSearchResults
# data = api.GetSearchResults(zwsid, address, zipcode)
# pp.pprint(data.get_dict())


# This appears to return the same as .GetSearchResults()
# detail_data = api.GetZEstimate(zwsid, data.zpid)
# pp.pprint(detail_data.get_dict())

{   'extended_data': {   'bathrooms': None,
                         'bedrooms': None,
                         'complete': False,
                         'finished_sqft': None,
                         'fips_county': None,
                         'last_sold_date': None,
                         'last_sold_price': None,
                         'lot_size_sqft': None,
                         'tax_assessment': None,
                         'tax_assessment_year': None,
                         'usecode': None,
                         'year_built': None},
    'full_address': {   'city': 'AUSTIN',
                        'latitude': '30.342641',
                        'longitude': '-97.718579',
                        'state': 'TX',
                        'street': '929 Cavalier Ln',
                        'zipcode': '78757'},
    'links': {   'comparables': 'http://www.zillow.com/homes/comps/119621219_zpid/',
                 'graphs_and_data': 'http://www.zillow.com/homedetails/92

## Calculate Value per Sqft

In [ ]:
# Seth

#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]
